# 1. Cài đặt thuật toán tiến trước, thuật toán Viterbi, và thuật toán Baum-Welch.

In [81]:
import numpy as np
import pandas as pd

## 1.1. Thuật toán tiến trước

[https://github.com/adeveloperdiary/HiddenMarkovModel/blob/master/part2/forward.py](https://github.com/adeveloperdiary/HiddenMarkovModel/blob/master/part2/forward.py)

In [82]:
def forward_algorithm(observation, transition_prob, emission_prob, initial_distribution, vocabulary):
    alpha = np.zeros((observation.shape[0], transition_prob.shape[0]))
    
    id = np.where(vocabulary == observation[0])

    alpha[0, :] = initial_distribution * emission_prob[:, id[0][0]]

    for i in range(1, observation.shape[0]):
        for j in range(transition_prob.shape[0]):
            id = np.where(vocabulary == observation[i])
            alpha[i, j] = alpha[i - 1].dot(transition_prob[:, j]) * emission_prob[j, id[0][0]]

    return alpha

## 1.2. Thuật toán Viterbi

[https://github.com/adeveloperdiary/HiddenMarkovModel/blob/master/part4/Viterbi.py](https://github.com/adeveloperdiary/HiddenMarkovModel/blob/master/part4/Viterbi.py)

In [83]:
def viterbi_algorithm(observation, transition_prob, emission_prob, initial_distribution, vocabulary):
    T = observation.shape[0] 
    M = transition_prob.shape[0] 

    omega = np.zeros((T, M))
    id = np.where(vocabulary == observation[0])
    
    omega[0, :] = np.log(initial_distribution * emission_prob[:, id[0][0]])

    prev = np.zeros((T - 1, M))

    for i in range(1, T):
        for j in range(M):
            
            # find the index of the observation in the vocabulary
            id = np.where(vocabulary == observation[i])

            # the same as forward probability
            probability = omega[i - 1] + np.log(transition_prob[:, j]) + np.log(emission_prob[j, id[0][0]])

            # the most probable state given previous state at time i    (1)
            prev[i - 1, j] = np.argmax(probability)

            # the probability of the most probable state                (2)
            omega[i, j] = np.max(probability)

    # path array
    path = np.zeros(T)

    # the most probable state at the last time step
    last_state = np.argmax(omega[T - 1, :])
    path[0] = last_state

    backtrack_index = 1

    for i in range(T - 2, -1, -1):
        path[backtrack_index] = prev[i, int(last_state)]
        last_state = prev[i, int(last_state)]
        backtrack_index += 1

    # flip the path array
    path = np.flip(path, axis = 0)

    # return path

    # convert numeric values to actual hidden states
    result = []

    for p in path:
        if p == 0:
            result.append("A")
        else:
            result.append("B")
    
    return result


## 1.3. Thuật toán Baum - Welch

In [84]:
def backward_algorithm(observation, transition_prob, emission_prob, vocabulary):
    beta = np.zeros((observation.shape[0], transition_prob.shape[0]))
    beta[observation.shape[0] - 1] = np.ones(transition_prob.shape[0])

    # loop backwards from t - 2 to 0
    for i in range(observation.shape[0] -2, -1, -1):
        for j in range(transition_prob.shape[0]):
            # find the index of the observation in the vocabulary
            id = np.where(vocabulary == observation[i + 1])

            beta[i, j] = (beta[i + 1] * emission_prob[:, id[0][0]]).dot(transition_prob[j, :])
    
    return beta

In [85]:
def baum_welch_algorithm(observation, transition_prob, emission_probability, initial_distribution, vocabulary, n_iter=100):
    M = transition_prob.shape[0]
    T = len(observation)

    for n in range(n_iter):
        alpha = forward_algorithm(observation, transition_prob, emission_probability, initial_distribution, vocabulary)
        beta = backward_algorithm(observation, transition_prob, emission_probability, vocabulary)

        xi = np.zeros((M, M, T - 1))
        for t in range(T - 1):
            id = np.where(vocabulary == observation[t + 1])

            denominator = np.dot(np.dot(alpha[t, :].T, transition_prob) * emission_probability[:, id[0][0]].T, beta[t + 1, :])
            for i in range(M):
                numerator = alpha[t, i] * transition_prob[i, :] * emission_probability[:, id[0][0]].T * beta[t + 1, :].T
                xi[i, :, t] = numerator / denominator

        gamma = np.sum(xi, axis=1)
        transition_prob = np.sum(xi, 2) / np.sum(gamma, axis=1).reshape((-1, 1))

        # Add additional T'th element in gamma
        gamma = np.hstack((gamma, np.sum(xi[:, :, T - 2], axis=0).reshape((-1, 1))))

        K = emission_probability.shape[1]
        denominator = np.sum(gamma, axis=1)
        for l in range(K):
            emission_probability[:, l] = np.sum(gamma[:, observation == l], axis=1)

        emission_probability = np.divide(emission_probability, denominator.reshape((-1, 1)))

    return (transition_prob, emission_probability)

## 1.4 Testing

In [86]:
data = pd.read_csv('data.csv')
obs = data['Visible'].values

# transition probabilities
trans_prob = np.ones((2, 2))
trans_prob = trans_prob / np.sum(trans_prob, axis = 1)

# emission probabilities
emiss_prob = np.array(((1, 3, 5), (2, 4, 6)))
emiss_prob = emiss_prob / np.sum(emiss_prob, axis = 1).reshape((-1, 1))

# equal probabilities for the initial distribution
init_dist = np.array((0.5, 0.5))

trans_prob, emiss_prob = baum_welch_algorithm(obs, trans_prob, emiss_prob, init_dist, np.array((0, 1, 2)))

result = viterbi_algorithm(obs, trans_prob, emiss_prob, init_dist, np.array((0, 1, 2)))

f = open("result.txt", "w")
for row in result:
    f.write(row + ' ')
f.close()

# 2. Bài toán: 
Khi làm quản trò, anh Huy thường sử dụng 2 viên xúc xác khác nhau. Viên đầu tiên là một viên xúc xắc cân bằng, mọi mặt đều có cùng xác suất. Viên thứ hai là một viên xúc xắc lỗi, khi tung sẽ có 50% xác suất ra mặt số 6 và 10% xác suất ra mỗi mặt còn lại. Mỗi lần tung, anh sẽ chọn 1 trong 2 viên xúc xắc này để tung. Người chơi không thể biết anh đã tung viên nào, chỉ biết được lần tung đó ra mặt nào. Ngoài ra, nếu ở lần tung này, anh Huy sử dụng viên xúc xắc cân bằng, thì có 80% khả năng anh sẽ tiếp tục sử dụng viên xúc xắc này cho lần tung tiếp theo (20% còn lại anh sẽ đổi sang dùng viên lỗi). Con số này là 30% đối với viên lỗi (70% đổi sang dùng viên cân bằng)

## a) Mô hình hóa tình huống trên bằng một mô hình Markov ẩn. Cho biết các tham số của mô hình này.
Mô hình Markov ẩn được xây dựng:
- Tập trạng thái Q = {q0: cân bằng, q1: lỗi}.
- Ma trận chuyển trạng thái A = [[0.8, 0.2], [0.7, 0.3]] (theo thứ tự a00, a01, a10, a11).
- Tập quan sát O gồm các trạng thái được lấy từ tập V = {1, 2, 3, 4, 5, 6}.
- Ma trận B (các giá trị observation likelihoods): 
[[P(1|q0) = 1/6, P(2|q0) = 1/6, P(3|q0) = 1/6, P(4|q0) = 1/6, P(5|q0) = 1/6, P(6|q0) = 1/6],
 [P(1|q1) = 0.1, P(2|q1) = 0.1, P(3|q1) = 0.1, P(4|q1) = 0.1, P(5|q1) = 0.1, P(6|q1) = 0.5]]
- Phân phối ban đầu Pi = [0.5, 0.5]

## b) Sinh ngẫu nhiên một chuỗi T = 100 lần tung theo đúng mô tả trên.

In [87]:
import random as rd

'A is the transition probability matrix'
A = [[0.8, 0.2], [0.7, 0.3]]

'B is the observation likelihoods matrix'
B = [[1/6, 1/6, 1/6, 1/6, 1/6, 1/6],[.1, .1, .1, .1, .1, .5]]

'D is the number on the faces of the dice'
D = [1,2,3,4,5,6]

'initial distribution'
Pi = np.array((0.5, 0.5))

def generate(T: int):
    dice = rd.choice([0,1])     # choose a random dice with equal probability 0.5
    res = list()
    dices = list()
    for i in range(0, T):
        dices.append(dice)
        temp1 = rd.choices(D, B[dice])
        res.append(temp1[0])
        temp2 = rd.choices([0,1], A[dice])
        dice = temp2[0]
    print(dices)
    return res

'Generate a sequence of T = 100 observations'
gen = np.array(generate(100))
print(gen)

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[6 6 5 3 2 5 4 2 6 6 2 2 2 6 1 4 5 3 4 1 5 3 6 6 3 3 4 2 4 1 6 6 1 6 5 2 1
 2 1 2 6 3 4 6 3 3 1 2 6 6 6 1 6 3 2 6 2 6 1 3 1 6 6 6 6 3 2 2 1 6 4 1 6 2
 5 6 5 5 6 4 3 6 6 1 4 1 6 3 4 3 6 5 2 6 6 1 1 6 1 6]


## c) Sử dụng thuật toán Viterbi để dự đoán viên xúc xắc được dùng cho mỗi lần tung. Độ chính xác của dự đoán này là bao nhiêu? Hãy lặp lại thí nghiệm này nhiều lần nếu cần thiết. Báo cáo và nhận xét kết quả thu được.

In [88]:
res1 = viterbi_algorithm(gen, np.array(A), np.array(B), Pi, np.array(D))
print(res1)

['B', 'B', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A']


## d) Giả sử bạn là một người chơi, hãy sử dụng thuật toán Baum-Welch để ước lượng các tham số cho mô hình Markov ẩn. Hãy lặp lại thí nghiệm nhiều lần nếu cần thiết. Báo cáo và nhận xét kết quả thu được.

In [89]:
# transition probabilities
A1 = np.ones((2, 2))
A1 = A1 / np.sum(A1, axis = 1)

# emission probabilities
B1 = np.array(((1, 3, 5, 7, 9, 11), (2, 4, 6, 8, 10, 12)))
B1 = B1 / np.sum(B1, axis = 1).reshape((-1, 1))

A1, B1 = baum_welch_algorithm(gen, A1, B1, Pi, np.array(D))
print(A1)
print(B1)

C:\Users\DELL\AppData\Local\Temp/ipykernel_16544/2692237171.py:16: RuntimeWarning: invalid value encountered in true_divide
  xi[i, :, t] = numerator / denominator


[[nan nan]
 [nan nan]]
[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]
